In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from dataset import pca_data_loader
from tqdm import tqdm

In [2]:
# 加载数据
train_loader, test_loader = pca_data_loader()

# 设置设备（如果有 GPU 可用则使用 GPU）
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [3]:
# 定义模型（两层全连接层：10 输入，n 隐藏单元，10 输出）
class SimpleMLPWithHidden(nn.Module):
    def __init__(self, hidden_units=50):
        super(SimpleMLPWithHidden, self).__init__()
        self.fc1 = nn.Linear(10, hidden_units)
        self.fc2 = nn.Linear(hidden_units, 10)

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        return x

In [4]:
# 初始化模型、损失函数和优化器
hidden_units = 50  # 隐藏层单元数量，可根据需要调整
model = SimpleMLPWithHidden(hidden_units).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [5]:
# 训练模型
epochs = 10
with tqdm(total=epochs * len(train_loader), desc="Training Progress", unit='batch') as pbar:
    for epoch in range(epochs):
        model.train()
        running_loss = 0.0
        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)

            # 前向传播
            outputs = model(inputs)
            loss = criterion(outputs, labels)

            # 反向传播和优化
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            running_loss += loss.item()
            pbar.update(1)

Training Progress: 100%|██████████| 9380/9380 [00:20<00:00, 468.16batch/s]


In [6]:
# 在测试集上评估模型
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for inputs, labels in test_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model(inputs)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

accuracy = 100 * correct / total
print(f"Test Accuracy: {accuracy:.2f}%")

Test Accuracy: 80.50%
